In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import networkx as nx

from itertools import tee

In [3]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [4]:
infilepath = r'/mnt/sda1/Users/Work/Transport Systems Catapult/MERGE - Documents/Models/MATSim/MATSim/Synthetic Pop/Building/inputs/ODs_purpose_mode_timeperiod.csv'

In [5]:
odtemp = pd.read_csv(infilepath, nrows=10)

FileNotFoundError: File b'/mnt/sda1/Users/Work/Transport Systems Catapult/MERGE - Documents/Models/MATSim/MATSim/Synthetic Pop/Building/inputs/ODs_purpose_mode_timeperiod.csv' does not exist

In [73]:
dtypes_keys = list(odtemp.columns)
dtypes_vals = ['category', 'category', 'category',
               'category', 'category', 'float']

dtypes = dict(zip(dtypes_keys, dtypes_vals))

od = pd.read_csv(infilepath, index_col=list(range(0, 5)))
#od.sort_index(inplace=True)

In [74]:
od.head(2)

Trips
orig_zoneID dest_zoneID Purpose Mode TimePeriod           
E02000001   E02000001   HBO     Bus  AM          20.572919
            E02000576   HBO     Bus  AM           3.849908

In [75]:
d_grp_purp = {'HBW': 'HB', 'HBO': 'HB', 'NHB': 'NHB'}
grp_keys = [None, None, d_grp_purp, None, None]

odg = od.groupby(grp_keys, level=od.index.names).sum()

In [76]:
odg.reset_index(inplace=True)
odg.Purpose = odg.Purpose.astype('category')

In [77]:
odg.head(2)

,orig_zoneID,dest_zoneID,Purpose,Mode,TimePeriod,Trips
0,E02000001,E02000001,HB,Bus,AM,93.610267
1,E02000001,E02000001,HB,Bus,IP,189.761532


for c in odg:
    if odg[c].dtype.name == 'category':
        odg[c] = odg[c].cat.codes

In [78]:
odg.set_index(['orig_zoneID', 'dest_zoneID', 'Purpose', 'Mode', 'TimePeriod'], inplace=True)

In [79]:
od_car = odg.xs('CarDriver', level='Mode')

In [80]:
od_car.reset_index(inplace=True)

for c in od_car:
    if od_car[c].dtype.name == 'category':
        od_car[c] = od_car[c].cat.codes

In [81]:
# Reduce
od_car_red = od_car[od_car.Trips >= 20]

In [82]:
G = nx.from_pandas_dataframe(od_car_red, 'orig_zoneID', 'dest_zoneID',
                             ['Purpose', 'TimePeriod'], nx.MultiDiGraph())

In [83]:
G.number_of_nodes()

912

In [84]:
G.number_of_edges()

39208

%time sum(1 for _ in potential_cycles)

In [89]:
potential_cycles = nx.simple_cycles(G, 5)

In [106]:
node_chain = next(potential_cycles)

In [107]:
node_chain

['E02000490', 'E02000493', 'E02000492', 'E02000491', 'E02000489']

In [91]:
G.get_edge_data(a[0], a[1])

{0: {'Purpose': 'HB', 'TimePeriod': 'AM'},
 1: {'Purpose': 'HB', 'TimePeriod': 'IP'},
 2: {'Purpose': 'HB', 'TimePeriod': 'OP'},
 3: {'Purpose': 'HB', 'TimePeriod': 'PM'},
 4: {'Purpose': 'NHB', 'TimePeriod': 'IP'}}

In [92]:
G.get_edge_data(a[1], a[2])

{0: {'Purpose': 'HB', 'TimePeriod': 'IP'},
 1: {'Purpose': 'HB', 'TimePeriod': 'OP'},
 2: {'Purpose': 'HB', 'TimePeriod': 'PM'}}

In [103]:
G.get_edge_data(a[2], a[3])

{0: {'Purpose': 'HB', 'TimePeriod': 'AM'},
 1: {'Purpose': 'HB', 'TimePeriod': 'IP'},
 2: {'Purpose': 'HB', 'TimePeriod': 'OP'},
 3: {'Purpose': 'HB', 'TimePeriod': 'PM'}}

In [ ]:
# First check if intermediate legs contain NHB trips
def nhb_in_interm_legs(G, node_chain):
    if len(node_chain) > 2:
        mid_node_chain = node_chain[1:-1]

        for a, b in pairwise(mid_node_chain):
            edges = G.get_edge_data(a, b)

            contains_nhb = False
            for k, edge in edges.items():
                if edge['Purpose'] == 'NHB':
                    contains_nhb = True
                    break

            if not contains_nhb:
                return False
        
        return True
            

In [ ]:


for a, b in pairwise(node_chain):
    